# **Data augmentation: unsing ChatGPT**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np

import random

!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI

import os

os.environ["OPENAI_API_KEY"] = "..." # put your key here

## **Load the data**

Load train set:

In [ ]:
f_in = open("drive/MyDrive/data/7-classes/train_set_paragraphs.pkl","rb")
#f_in = open("drive/MyDrive/data/7-classes/train_set_sentences.pkl","rb")

data_train = pickle.load(f_in)

f_in.close()

Classes to augment:

In [ ]:
data_train_positive = [data_train[i][1] for i in range(len(data_train)) if data_train[i][2] == 0]
#data_train_positive = [data_train[i][1] for i in range(len(data_train)) if (data_train[i][2] == 0) or (data_train[i][2] == 1) or (data_train[i][2] == 2)]
#data_train_positive = [data_train[i][1] for i in range(len(data_train)) if (data_train[i][2] == 0) or (data_train[i][2] == 6)]

In [ ]:
data_train_positive[0:5]

['Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)\n\nBlock “strengths and challenges”\n\nPlease describe your challenges regarding the valorization of your agricultural by-products from apple-/tomato-/grape-/potato processing /brewer’s grains  as of today, e.g.:\n\nSmells: Same',
 'Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)\n\nBlock “current structures”\n\nHow satisfied are you with your current valorization of apple-/tomato-/grape-/potato processing/brewer’s grains? Why?\n\nWe have got our own biogas plant since 1980 and we are therefore very experienced and the valorisation of potato waste is nothing new for us. We recently added a processing plant for the processing to fossil gas quality. This and the recent developments have helped us to turn our waste valorisation operation into a zero losses zero profit operation. So 

In [ ]:
len(data_train_positive)

101

## **Perform augmentation**

### **Method 1: new segment based on a current segment**

Define model:

In [ ]:
client = OpenAI()

Experiment (augmentation) name:

In [ ]:
experiment = '2'

Perform augmentation:

In [ ]:
new_training_set = []

for i in range(len(data_train_positive)):
  # querry 1:
  #request = 'Generate new textual segment using an example: "'+data_train_positive[i]+'"'
  # querry 2:
  request = 'A textual segment has a name of a questionnaire, name of the block, the question and response. Generate new textual segment using an example, but keep the same questionnaire, name of the block and the question: "'+data_train_positive[i]+'"'
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": request,
          }
      ],
      model="gpt-3.5-turbo-0301",
  )
  new_training_set.append(chat_completion.choices[0].message.content)

  if i % 10 == 0:
    print("Process",i,"segment")

Process 0 segment
Process 10 segment
Process 20 segment
Process 30 segment
Process 40 segment
Process 50 segment
Process 60 segment
Process 70 segment
Process 80 segment
Process 90 segment
Process 100 segment


Generated phrases:

In [ ]:
new_training_set[0:5]

['Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)\n\nBlock “strengths and challenges”\n\nPlease describe your strengths and challenges regarding the valorization of your agricultural by-products from apple-/tomato-/grape-/potato processing /brewer’s grains  as of today, e.g.:\n\nChallenge: Smells are a big challenge for me.',
 'Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)\n\nBlock “current structures”\n\nHow satisfied are you with your current valorization of apple-/tomato-/grape-/potato processing/brewer’s grains? Why?\n\nOne farmer in the survey reported being very satisfied with his current valorization of tomato processing waste. He explained that his family-owned farm has been using the waste to create fertilizer for years, meaning they have a well-established method for dealing with this waste product. In addition, the 

Create new segments:

In [ ]:
data_new = [(-1,i,cls) for i in new_training_set]

data_augmented = data_train + data_new

Some stats:

In [ ]:
len(data_new)

101

In [ ]:
len(data_augmented)

525

Save results:

In [ ]:
f_out = open("drive/MyDrive/data/7-classes/train_set_paragraphs_augm-"+experiment+".pkl","wb")
#f_out = open("drive/MyDrive/data/7-classes/train_set_sentences_augm-"+experiment+".pkl","wb")

pickle.dump(data_augmented,f_out)

f_out.close()

### **Method 1: new segment based on a current segment**

Experiment (augmentation) name:

In [ ]:
experiment = '3'

Perform augmentation:

In [ ]:
new_training_set = []

count = 0
while(len(new_training_set)) < len(data_train_positive):
  segment_ids = random.sample(range(0,len(data_train_positive)),3)

  phrases_str = ""
  for i in range(3):
    phrases_str += 'Example '+str(i+1)+': "'+ data_train_positive[segment_ids[i]] + '"\n\n\n'

  request = 'Generate new textual segment using the following examples: "'+phrases_str.strip()+'"'

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": request,
          }
      ],
      model="gpt-3.5-turbo-0301",
  )
  new_example = chat_completion.choices[0].message.content

  if "Example 4: " in new_example:
    new_training_set.append(new_example.split('Example 4: ')[1].strip('"'))
    count += 1

    if count % 10 == 0:
      print("Process",count,"segment")

Process 0 segment
Process 10 segment
Process 10 segment
Process 20 segment
Process 20 segment
Process 20 segment
Process 30 segment
Process 30 segment
Process 30 segment
Process 40 segment
Process 50 segment
Process 50 segment
Process 50 segment
Process 60 segment
Process 60 segment
Process 70 segment
Process 80 segment
Process 80 segment
Process 90 segment
Process 100 segment
Process 100 segment
Process 100 segment
Process 100 segment


Generated phrases:

In [ ]:
new_training_set[0:5]

['Questionnaire for arable farmers/ horticulture businesses as users of the end-product PHA applications (mulch films etc.)\n\nBlock “Usage and experience”\n\nHow frequently do you currently use PHA applications such as mulch films in your farming practices? Please select one:\n\n- Daily\n- Several times per week\n- Once per week\n- Less often than once per week, but regularly\n- Rarely\n- Never\n\n\nExample 5: "Questionnaire for arable farmers/ horticulture businesses as users of the end-product PHA applications (mulch films etc.)\n\nBlock “Feedback on PHA products”\n\nAre you satisfied with the quality of PHA applications such as mulch films? Please rate your satisfaction on a scale of 1-5, with 1 being very unsatisfied and 5 being very satisfied.',
 'Questionnaire for arable farmers/ horticulture businesses as users of the end-product PHA applications (mulch films etc.)\n\nBlock: Advantages and challenges\n\nWhat are the advantages of using bio-based and bio-degradable mulch films, 

Create new segments:

In [ ]:
data_new = [(-1,i,cls) for i in new_training_set]

data_augmented = data_train + data_new

Some stats:

In [ ]:
len(data_new)

101

In [ ]:
len(data_augmented)

525

Save results:

In [ ]:
f_out = open("drive/MyDrive/data/7-classes/train_set_paragraphs_augm-"+experiment+".pkl","wb")
#f_out = open("drive/MyDrive/data/7-classes/train_set_sentences_augm-"+experiment+".pkl","wb")

pickle.dump(data_augmented,f_out)

f_out.close()